# Handwritten Text Generation
This notebook covers the task of generating handwritten-like text using a character-level recurrent neural network (RNN).
We will use a sample handwritten text dataset.
The notebook includes dataset download, preprocessing, model training, and text generation.

In [ ]:
# Install necessary libraries
!pip install -q tensorflow numpy


In [ ]:
# Download dataset
import os
import urllib.request

dataset_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
dataset_path = 'input.txt'

if not os.path.exists(dataset_path):
    print('Downloading dataset...')
    urllib.request.urlretrieve(dataset_url, dataset_path)
    print('Download complete.')
else:
    print('Dataset already exists.')


In [ ]:
# Load dataset
with open(dataset_path, 'r', encoding='utf-8') as f:
    text = f.read()
print(f'Dataset length: {len(text)} characters')

## Data Preprocessing
- Create character to index and index to character mappings
- Prepare input sequences and targets for training


In [ ]:
import numpy as np

chars = sorted(list(set(text)))
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

seq_length = 100
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i:i+seq_length])
    next_chars.append(text[i+seq_length])

print(f'Number of sequences: {len(sentences)}')

In [ ]:
# Vectorize sequences
X = np.zeros((len(sentences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool_)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_idx[char]] = 1
    y[i, char_to_idx[next_chars[i]]] = 1

## Model Building
We will build a simple LSTM model for character-level text generation.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(128, input_shape=(seq_length, len(chars))),
    Dense(len(chars), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

## Model Training
Train the model for a few epochs.

In [ ]:
model.fit(X, y, batch_size=128, epochs=10)

## Text Generation
Generate text using the trained model.

In [ ]:
import random

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(length=400, temperature=1.0):
    start_index = random.randint(0, len(text) - seq_length - 1)
    generated = ''
    sentence = text[start_index:start_index + seq_length]
    generated += sentence
    for i in range(length):
        x_pred = np.zeros((1, seq_length, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_idx[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = idx_to_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
    return generated

# Generate sample text
print(generate_text())